In [6]:
#Importing packages and checking GPU
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split

if torch.cuda.is_available():
	print("PyTorch is using the GPU")
	GPUCount = torch.cuda.device_count()
	print(f"Found {GPUCount} GPUs")

	for i in range(GPUCount):
		print(f"GPU {i} found: {torch.cuda.get_device_name(i)}")

	device = torch.device("cuda:0")
else:
	print("PyTorch is using the CPU")
	device = torch.device("cpu")

print(f"Selected Device: {device}")

PyTorch is using the GPU
Found 1 GPUs
GPU 0 found: NVIDIA GeForce RTX 5070 Laptop GPU
Selected Device: cuda:0


In [7]:
#Load BuiltIn dataset
Transform = transforms.Compose(
	[transforms.ToTensor(),
  transforms.Normalize((0.5,),(0.5,))])
Transform = transforms.Compose(
	[transforms.ToTensor(),
  transforms.Normalize((0.5,),(0.5,))])

TrainSet = torchvision.datasets.EMNIST(root='./data',train=True,download=True,transform=Transform,split="letters",target_transform=lambda y:y-1)
TrainLoader = DataLoader(TrainSet, batch_size=128, shuffle=True)

TestSet = torchvision.datasets.EMNIST(root='./data',train=False,download=True,transform=Transform,split="letters",target_transform=lambda y:y-1)
TestLoader = DataLoader(TestSet, batch_size=128, shuffle=False)

In [8]:
#Creating Model

#Define architecture
class EMNISTModel(nn.Module):
	def __init__(self):
		super(EMNISTModel,self).__init__()

		#Layer1
		self.conv1 = nn.Conv2d(in_channels=1,out_channels=32,kernel_size=3)#Size does not change
		self.relu1 = nn.ReLU()
		self.pool1 = nn.MaxPool2d(kernel_size=2)#Size halves into 14x14

		#Layer2
		self.conv2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=3)
		self.relu2 = nn.ReLU()
		self.pool2 = nn.MaxPool2d(kernel_size=2)#Size halves into 7x7

		#Layer3
		self.conv3 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3)

		#Layer4
		self.flatten = nn.Flatten()

		#Layer5
		self.fc1 = nn.Linear(in_features=64*3*3,out_features=64)
		self.relu3 = nn.ReLU()

		#Layer6
		self.fc2 = nn.Linear(in_features=64,out_features=26)

	def forward(self,x):

		#Pass through Layer1
		x = self.pool1(self.relu1(self.conv1(x)))

		#Pass through Layer2
		x = self.pool2(self.relu2(self.conv2(x)))

		#Pass through Layer3
		x = self.conv3(x)

		#Pass through Layer4
		x = self.flatten(x)

		#Pass through Layer5
		x = self.relu3(self.fc1(x))

		#Pass through Layer6
		x = self.fc2(x)

		#Return Prediction
		return x

#Create and print summary	
print("Creating EMNIST Model...")
model = EMNISTModel().to(device)
print("Model Created")

print("Model Summary: ")
summary(model,input_size=(1,28,28))

#Compile model
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

Creating EMNIST Model...
Model Created
Model Summary: 
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
              ReLU-2           [-1, 32, 26, 26]               0
         MaxPool2d-3           [-1, 32, 13, 13]               0
            Conv2d-4           [-1, 64, 11, 11]          18,496
              ReLU-5           [-1, 64, 11, 11]               0
         MaxPool2d-6             [-1, 64, 5, 5]               0
            Conv2d-7             [-1, 64, 3, 3]          36,928
           Flatten-8                  [-1, 576]               0
            Linear-9                   [-1, 64]          36,928
             ReLU-10                   [-1, 64]               0
           Linear-11                   [-1, 26]           1,690
Total params: 94,362
Trainable params: 94,362
Non-trainable params: 0
------------------------------------------

In [9]:
#Training and saving the model

print("Training Model...")
epochs = 30
patience = 5
counter = 0
MinValidationLoss = float('inf')

for epoch in range(epochs):
	#Train Loop

	#Set model to training mode
	model.train()
	TrainLoss = 0.0

	for i,data in enumerate(TrainLoader,0):
		inputs,labels = data[0].to(device),data[1].to(device)

		#Zero the parameter gradients
		optimizer.zero_grad()

		#Forward pass
		outputs = model(inputs)

		#Calculate loss
		loss = loss_function(outputs,labels)

		#Backward pass
		loss.backward()

		#Update weights
		optimizer.step()

		#Update loss
		TrainLoss += loss.item()
	
	#Validation Loop
	model.eval()
	ValidationLoss = 0.0
	correct = 0
	total = 0

	with torch.no_grad():
		for data in TestLoader:
			images,labels = data[0].to(device),data[1].to(device)
			outputs = model(images)
			loss = loss_function(outputs,labels)
			ValidationLoss += loss.item()

			_,predicted = torch.max(outputs.data,1)
			total += labels.size(0)
			correct += (predicted == labels).sum().item()

		acc = 100*correct/total
		print(f"Epoch: {epoch+1}/{epochs} | "
			f"Training Loss: {TrainLoss/len(TrainLoader):.3f} | "
			f"Validation Loss: {ValidationLoss / len(TestLoader):.3f} | "
			f"Accuracy: {acc:.3f}%")
		
		if ValidationLoss < MinValidationLoss:
			MinValidationLoss = ValidationLoss
			counter = 0
			torch.save(model.state_dict(),"EMNISTModel.pth")
			print(f"Validation loss has improved at epoch {epoch+1}. Model Saved!")
		else:
			counter += 1
			print(f"Validation loss has not improved. Patience:{counter}/{patience}")
		
		if counter >= patience:
			print("Early stopping triggered!")
			break
		
print("Training Complete")


Training Model...
Epoch: 1/30 | Training Loss: 0.557 | Validation Loss: 0.291 | Accuracy: 90.481%
Validation loss has improved at epoch 1. Model Saved!
Epoch: 2/30 | Training Loss: 0.264 | Validation Loss: 0.249 | Accuracy: 91.644%
Validation loss has improved at epoch 2. Model Saved!
Epoch: 3/30 | Training Loss: 0.223 | Validation Loss: 0.235 | Accuracy: 92.514%
Validation loss has improved at epoch 3. Model Saved!
Epoch: 4/30 | Training Loss: 0.202 | Validation Loss: 0.210 | Accuracy: 93.188%
Validation loss has improved at epoch 4. Model Saved!
Epoch: 5/30 | Training Loss: 0.185 | Validation Loss: 0.218 | Accuracy: 92.769%
Validation loss has not improved. Patience:1/5
Epoch: 6/30 | Training Loss: 0.171 | Validation Loss: 0.204 | Accuracy: 93.346%
Validation loss has improved at epoch 6. Model Saved!
Epoch: 7/30 | Training Loss: 0.161 | Validation Loss: 0.196 | Accuracy: 93.394%
Validation loss has improved at epoch 7. Model Saved!
Epoch: 8/30 | Training Loss: 0.154 | Validation Los

In [11]:
#Testing Model

print("Testing model with inbuilt dataset...")

model.load_state_dict(torch.load("EMNISTModel.pth"))

TestLoss = 0.0
correct = 0
total = 0

with torch.no_grad():
	for images, labels in TestLoader:

		#Move data to device
		images,labels = images.to(device),labels.to(device)

		#Forward pass
		outputs = model(images)

		#Calculate loss
		loss = loss_function(outputs,labels)
		TestLoss += loss.item()

		#Get predicted class
		_,predicted = torch.max(outputs.data,1)

		#Update total and correct counts
		total += labels.size(0)
		correct += (predicted == labels).sum().item()
	
#Test Results

FinalLoss = TestLoss/len(TestLoader)
FinalAcc = 100*correct/total

print(f"Test Accuracy: {FinalAcc:.3f}%")
print(f"Test Loss: {FinalLoss:.3f}")

Testing model with inbuilt dataset...
Test Accuracy: 93.784%
Test Loss: 0.191
